<a href="https://colab.research.google.com/github/Mohan0332/Capstone/blob/main/features%20extraction%20using%20InceptioV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###InceptionV3

In [10]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Define the directories containing the training and testing data
train_dir = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB/train'
test_dir = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB/test'

# Define the InceptionV3 model
inception = InceptionV3(include_top=False, pooling='avg')

# Define a function to extract features from a given image file
def extract_features(file_path):
    img = image.load_img(file_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    features = inception.predict(x)
    return features

# Define a function to extract features from a directory of images
def extract_features_from_dir(dir_path):
    features = []
    labels = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith(('.jpg','.png')):
                file_path = os.path.join(root, file)
                label = os.path.basename(root)
                feature = extract_features(file_path)
                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)

# Extract features from the training and testing directories
train_features, train_labels = extract_features_from_dir(train_dir)
test_features, test_labels = extract_features_from_dir(test_dir)



1/1 [==============================] - 0s 244ms/step


In [11]:
# Flatten the features for use in a linear SVM
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

In [14]:
train_features.shape

(490, 1, 2048)

In [15]:
# Convert the labels to integers
train_labels = np.array([int(label) for label in train_labels])
test_labels = np.array([int(label) for label in test_labels])

# Train a linear SVM on the flattened features
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(train_features_flat, train_labels)


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [16]:
# Evaluate the SVM on the test data
accuracy = svm.score(test_features_flat, test_labels)
print('Accuracy:', accuracy)

Accuracy: 0.7368421052631579


In [ ]:
#Logistic Regression

In [17]:
# Train a linear SVM on the flattened features
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_features_flat, train_labels)

# Evaluate the SVM on the test data
accuracy = logreg.score(test_features_flat, test_labels)
print('Accuracy:', accuracy)

Accuracy: 0.7894736842105263


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
#Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
# Train a random forest classifier on the flattened features
rfc = RandomForestClassifier()
rfc.fit(train_features_flat, train_labels)

# Evaluate the random forest classifier on the test data
accuracy = rfc.score(test_features_flat, test_labels)
print('Accuracy:', accuracy)

Accuracy: 0.7192982456140351


In [21]:
#gradient Boosting

In [23]:
from sklearn.ensemble import GradientBoostingClassifier
# Train a Gradient Boosting Classifier on the flattened features
gb = GradientBoostingClassifier(n_estimators=100,max_depth=3,random_state=42)
gb.fit(train_features_flat, train_labels)

# Evaluate the Gradient Boosting Classifier on the test data
accuracy = gb.score(test_features_flat, test_labels)
print('Accuracy:', accuracy)




Accuracy: 0.6842105263157895


In [29]:
#MLP standalone

In [39]:
# Train a MLP classifier on the flattened features
mlp = MLPClassifier(hidden_layer_sizes=(1024,), max_iter=1000)
mlp.fit(train_features_flat, train_labels)

MLPClassifier(hidden_layer_sizes=(1024,), max_iter=1000)

In [40]:
# Evaluate the MLP classifier on the test data
accu = mlp.score(test_features_flat, test_labels)
print('Accuracy:', accu)

Accuracy: 0.8070175438596491


Ensemble

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

In [42]:
logreg = LogisticRegression()
svm = LinearSVC()
mlp = MLPClassifier()
voting_clf = VotingClassifier(estimators=[('lr', logreg),('mlp', mlp)], voting='hard')
voting_clf.fit(train_features_flat, train_labels)

# Evaluate the voting classifier on the test data
accuracy = voting_clf.score(test_features_flat, test_labels)
print('Accuracy:', accuracy)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8245614035087719


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
